<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/DRLforMultipleStocksTradingUsingFinRL/blob/main/FractionsTest3/3_9_fraksi2_TugasAkhirS1_Shinta_DRLforMultipleStocksTradingUsingFinRL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook ini dijalankan menggunakan email shinta.hanafia@gmail.com

# Deep Reinforcement Learning untuk Jual-Beli Saham (Dari Awal): Jual-beli Banyak Saham 

* **Pytorch Version** 



# Daftar Isi:

* [1. Pendeskrisian Tugas](#0)
* [2. Instalasi Paket Python](#1)
    * [2.1. Instalasi Paket](#1.1)    
    * [2.2. Daftar Paket Python](#1.2)
    * [2.3. Impot Paket](#1.3)
    * [2.4. Membuat Folder](#1.4)
* [3. Mengunduh Data](#2)
* [4. Memproses Data](#3)        
    * [4.1. IndiKator Teknikal](#3.1)
    * [4.2. Melakukan *Feature Engineering*](#3.2)
* [5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style](#4)  
    * [5.1. Pemisahan Data](#4.1)  
    * [5.2. Lingkungan untuk Pelatihan *(Training)*](#4.2)    
* [6. Melatih Agen DRL](#5)
    * [6.1. Agen 1: A2C](#5.1)
    * [6.2. Agen 2: PPO](#5.2)
    * [6.3. Agen 3: TD3](#5.3)
    * [6.4. Agen 4: SAC](#5.4)
    * [6.5. Agen 5: DDPG](#5.5)
* [7. Melakukan Jual-Beli](#6)
    * [7.1. Performa di Dalam Sampel](#6.1)
    * [7.2. Performa di Luar Sampel](#6.2)
    * [7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi](#6.3)
      * [7.3.1. Agen 1: A2C](#6.3.1)
      * [7.3.2. Agen 2: PPO](#6.3.2)
      * [7.3.3. Agen 3: TD3](#6.3.3)
      * [7.3.4. Agen 4: SAC](#6.3.4)
      * [7.3.5. Agen 5: DDPG](#6.3.5)
* [8. Performa Backtesting](#7)  
    * [8.1. Status BackTesting](#7.1)
    * [8.2. Gambaran BackTesting](#7.2)

<a id='0'></a>
# Bagian 1. Pendeskrisian Tugas

Agen DRL dilatih untuk melakukan jual-beli saham. Tugas tersebut dimodelkan sebagai Markov Decision Process (MDP), dengan fungsi dan tujuannya adalah memaksimalkan pengembalian pengembalian kumulatif *(cummulative return)* yang diharapkan.

Definisi state-action-reward pada algoritam DRL kasus kali ini adalah sebagai berikut:

* **State s**: Kondisi atau *state* mewakili persepsi agen tentang lingkungan pasar. Sama seperti *trader* manusia yang menganalisis berbagai informasi, agen juga secara pasif mengamati banyak fitur dan belajar dengan cara berinteraksi dengan lingkungan pasar (biasanya dengan menjalankan ulang data historis).

* **Tindakan atau *action* a**: Ruang aksi mencakup aksi atau tindakan yang dapat dilakukan agen di setiap status atau kondisi. Misalnya, a {−1, 0, 1}, -1 berarti menjual, 0 berarti menahan, dan 1 berarti membeli. Ketika suatu aksi mengoperasikan beberapa saham, maka, a {−k, ..., 1, 0, 1, ..., k}. Misalnya, "Beli
10 saham TLKM" atau "Jual 10 saham TLKM" maka masing-masing nilai a adalah 10 atau -10.

* **Fungsi *reward* atau imbalan r(s, a, s′)**: *Reward* adalah insentif bagi agen untuk mempelajari kebijakan yang lebih baik. Misalnya *reward* dapat berupa perubahan nilai portofolio saat mengambil a pada keadaan s dan tiba pada keadaan baru s', yaitu, r(s, a, s′) = v′-v, v′ mewakili nilai portofolio pada keadaan s′ dan v mewakili nilai portofolio pada keadaan s.

* **Environment atau lingkungan jual-beli**: saham penyusun indeks JII dengan tanggal sesuai periode pengujian yang diatur.


Data untuk studi kasus ini diperoleh dari Yahoo Finance API. Data berisi harga *Open-High-Low-Close* dan *Volume*.

<a id='1'></a>
# Bagian 2. Instalasi Paket Python

<a id='1.1'></a>
## 2.1. Instalasi Paket


In [1]:
# install finrl library
!pip install git+https://github.com/shintaroudlotulhanafia/FinRL.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/shintaroudlotulhanafia/FinRL.git to /tmp/pip-req-build-yonrp5po
  Running command git clone -q https://github.com/shintaroudlotulhanafia/FinRL.git /tmp/pip-req-build-yonrp5po
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-jk64urm7/pyfolio_fcc8f63ac0da4c368f7e15c5f9701bf4
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-jk64urm7/pyfolio_fcc8f63ac0da4c368f7e15c5f9701bf4
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-jk64urm7/elegantrl_b958c41f50b9473285937d25f752ca89
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-jk64urm7/elegantrl_b958c41f50b9473285937d25f752ca89
     |████████████████████████████████| 2.3 MB 38.7 MB/s 
     |████████████████████████████████| 234 kB 71.5 MB/s 
     |█████████████████████

<a id='1.2'></a>
## 2.2. Daftar Paket Python
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Impot Paket

* Modul Python **pandas** digunakan untuk menganalisis dan memanipulasi data.
* Modul Python **numpy** digunakan untuk memproses larik atau array.
* Modul Python **matplotlib** digunakan membuat visualisasi data dalam dua dimensi.
* Modul Python **matplotlib.pyplot** adalah kumpulan fungsi yang membuat matplotlib berfungsi seperti MATLAB.
* Modul Python **Datetime** menyediakan sejumlah fungsi untuk menangani tanggal, waktu, dan interval waktu. Date dan datetime adalah objek dalam Python, bukan string atau timestamps.
* **YahooDownloader** menyediakan metode untuk mengambil data saham harian dari API Keuangan Yahoo!
* **FeatureEngineer** menyediakan metode untuk preprocessing data harga saham
* **data_split** membagi dataset menjadi data pelatihan dan data pengujian berdasarkan tanggal
* **StockTradingEnv** Lingkungan perdagangan saham untuk OpenAI gym
* **DRLAgent** menyediakan implementasi untuk algoritma DRL
* **DataProcessor** memproses data menggunakan prosesor data terpadu
* **backtest_stats** menghitung statistik *backtesting*
* **backtest_plot** membuat dan menampilkan plot ringkasan laporan *backtesting*.
* **get_daily_return** 
* **get_baseline** mengunduh data berdasarkan suatu indeks pada periode waktu tertentu.
* Metode **sys.path.append()** digunakan untuk menambahkan jalur sementara. Dengan demikian, jalur tersebut akan valid untuk sebuah sesi, misalnya.
* Python **itertool** adalah modul yang menyediakan berbagai fungsi yang bekerja pada iterator untuk menghasilkan iterator yang kompleks. Modul ini berfungsi sebagai alat yang cepat dan hemat memori.

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


<a id='1.4'></a>
## 2.4. Membuat Folder

In [3]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Mengunduh Data

Yahoo Finance menyediakan data saham, berita keuangan, laporan keuangan, dll, secara gratis.
* FinRL *Library* menggunakan kelas **YahooDownloader** di FinRL-Meta untuk mengambil data melalui Yahoo Finance API
* Batas Pemanggilan: Menggunakan API Publik (tanpa autentikasi), Pengguna dibatasi hingga 2.000 permintaan per jam per IP (atau hingga total 48.000 permintaan per hari).

In [4]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2017-12-31'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2019-12-20'

In [5]:
#Data diperoleh dari idx.co.id diakses pada tanggal 27/08/2022
#Fraksi 1: x<Rp200,00
fraksi1 = ['KIJA.JK', 'LCGP.JK', 'LMPI.JK', 'LPKR.JK']

#Fraksi 2: Rp200,00<= x <Rp500,00
fraksi2 = ['BMTR.JK','BTON.JK','FORU.JK','GEMA.JK']

#Fraksi 3: Rp500,00<= x <Rp2.000,00
fraksi3 = ['AKRA.JK','BRPT.JK','KLBF.JK','MEDC.JK']

#Fraksi 4: Rp2000,00<= x <Rp5000,00
fraksi4 = ['JECC.JK', 'TMAS.JK', 'TPIA.JK', 'UNVR.JK']

#Fraksi 5: >=Rp5000,00
fraksi5 =['INCO.JK','INDF.JK','INTP.JK','UNTR.JK']

In [57]:
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = fraksi2).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (10912, 8)


In [58]:
df.nunique()

date      2728
open       874
high       895
low        893
close     1965
volume    4913
tic          4
day          5
dtype: int64

In [59]:
print(df['tic'].unique())

['BMTR.JK' 'BTON.JK' 'FORU.JK' 'GEMA.JK']


* **sort_values()** berfungsi mengurutkan bingkai data dalam urutan Ascending atau Descending dari kolom yang dilewati.
* **ignore_index** berfungsi untuk menentukan untuk mereset indeks mulai dari nol. Secara default disetel *false*.
* **head()** digunakan untuk mendapatkan n baris pertama.

In [60]:
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-05,185.00,188.00,180.00,165.136475,4,BMTR.JK,0
1,2009-01-05,83.75,83.75,83.75,79.111000,0,BTON.JK,0
2,2009-01-05,55.00,55.00,55.00,41.633144,10000,FORU.JK,0
3,2009-01-05,28.00,28.00,28.00,21.837465,15,GEMA.JK,0
4,2009-01-06,185.00,188.00,185.00,165.136475,362500,BMTR.JK,1


<a id='3'></a>
# Bagian 4: Memproses Data

Periksa data yang hilang dan melakukan *feature engineering* untuk mengubah data menjadi *state*.
* **Menambahkan indikator teknis**

  Dalam praktik jual-beli, berbagai informasi perlu diperhitungkan, seperti harga historis, kepemilikan saham saat ini, indikator teknis, dll. Indikator teknis yang dapat digunakan antara lain, MACD, RSI, CCI, ADX, Bollinger Bands, dll. 
* **Menambahkan indeks turbulensi**

  Risk-aversion mencerminkan seberapa berani investor melakukan jual-beli saham dengan risiko yang tinggi. Hal ini mempengaruhi strategi trading investor ketika menghadapi tingkat volatilitas pasar yang berbeda. Untuk mengendalikan risiko dalam skenario terburuk, seperti krisis keuangan tahun 1998, FinRL menggunakan indeks turbulensi yang mengukur fluktuasi harga aset yang ekstrem.

In [61]:
INDICATORS = [
    "macd",
    "rsi_30",
    "cci_30"
]

Membersihkan data mentah dari nilai-nilai yang hilang. Sehingga saham dengan data yang hilang dihapus dari daftar.

In [62]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (2759, 8)
Successfully added vix
Successfully added turbulence index


In [63]:
processed.nunique()

date           2651
open            872
high            893
low             893
close          1948
volume         4808
tic               4
day               5
macd          10414
rsi_30         6822
cci_30         9685
vix            1441
turbulence     2397
dtype: int64

In [64]:
list_ticker = processed["tic"].unique().tolist()
list_ticker

['BMTR.JK', 'BTON.JK', 'FORU.JK', 'GEMA.JK']

In [65]:
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
list_date

['2009-01-05',
 '2009-01-06',
 '2009-01-07',
 '2009-01-08',
 '2009-01-09',
 '2009-01-10',
 '2009-01-11',
 '2009-01-12',
 '2009-01-13',
 '2009-01-14',
 '2009-01-15',
 '2009-01-16',
 '2009-01-17',
 '2009-01-18',
 '2009-01-19',
 '2009-01-20',
 '2009-01-21',
 '2009-01-22',
 '2009-01-23',
 '2009-01-24',
 '2009-01-25',
 '2009-01-26',
 '2009-01-27',
 '2009-01-28',
 '2009-01-29',
 '2009-01-30',
 '2009-01-31',
 '2009-02-01',
 '2009-02-02',
 '2009-02-03',
 '2009-02-04',
 '2009-02-05',
 '2009-02-06',
 '2009-02-07',
 '2009-02-08',
 '2009-02-09',
 '2009-02-10',
 '2009-02-11',
 '2009-02-12',
 '2009-02-13',
 '2009-02-14',
 '2009-02-15',
 '2009-02-16',
 '2009-02-17',
 '2009-02-18',
 '2009-02-19',
 '2009-02-20',
 '2009-02-21',
 '2009-02-22',
 '2009-02-23',
 '2009-02-24',
 '2009-02-25',
 '2009-02-26',
 '2009-02-27',
 '2009-02-28',
 '2009-03-01',
 '2009-03-02',
 '2009-03-03',
 '2009-03-04',
 '2009-03-05',
 '2009-03-06',
 '2009-03-07',
 '2009-03-08',
 '2009-03-09',
 '2009-03-10',
 '2009-03-11',
 '2009-03-

In [66]:
combination = list(itertools.product(list_date,list_ticker))
combination

[('2009-01-05', 'BMTR.JK'),
 ('2009-01-05', 'BTON.JK'),
 ('2009-01-05', 'FORU.JK'),
 ('2009-01-05', 'GEMA.JK'),
 ('2009-01-06', 'BMTR.JK'),
 ('2009-01-06', 'BTON.JK'),
 ('2009-01-06', 'FORU.JK'),
 ('2009-01-06', 'GEMA.JK'),
 ('2009-01-07', 'BMTR.JK'),
 ('2009-01-07', 'BTON.JK'),
 ('2009-01-07', 'FORU.JK'),
 ('2009-01-07', 'GEMA.JK'),
 ('2009-01-08', 'BMTR.JK'),
 ('2009-01-08', 'BTON.JK'),
 ('2009-01-08', 'FORU.JK'),
 ('2009-01-08', 'GEMA.JK'),
 ('2009-01-09', 'BMTR.JK'),
 ('2009-01-09', 'BTON.JK'),
 ('2009-01-09', 'FORU.JK'),
 ('2009-01-09', 'GEMA.JK'),
 ('2009-01-10', 'BMTR.JK'),
 ('2009-01-10', 'BTON.JK'),
 ('2009-01-10', 'FORU.JK'),
 ('2009-01-10', 'GEMA.JK'),
 ('2009-01-11', 'BMTR.JK'),
 ('2009-01-11', 'BTON.JK'),
 ('2009-01-11', 'FORU.JK'),
 ('2009-01-11', 'GEMA.JK'),
 ('2009-01-12', 'BMTR.JK'),
 ('2009-01-12', 'BTON.JK'),
 ('2009-01-12', 'FORU.JK'),
 ('2009-01-12', 'GEMA.JK'),
 ('2009-01-13', 'BMTR.JK'),
 ('2009-01-13', 'BTON.JK'),
 ('2009-01-13', 'FORU.JK'),
 ('2009-01-13', 'GEM

In [67]:
processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136475,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837465,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136475,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,320.00,348.00,320.00,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038941
15996,2019-12-18,BMTR.JK,368.00,370.00,358.00,358.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,212.00,212.00,188.00,198.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,101.00,101.00,101.00,101.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


In [68]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136475,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837465,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136475,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,320.00,348.00,320.00,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038941
15996,2019-12-18,BMTR.JK,368.00,370.00,358.00,358.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,212.00,212.00,188.00,198.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,101.00,101.00,101.00,101.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


In [69]:
processed_full = processed_full.sort_values(['date','tic'])
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136475,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837465,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136475,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,320.00,348.00,320.00,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038941
15996,2019-12-18,BMTR.JK,368.00,370.00,358.00,358.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,212.00,212.00,188.00,198.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,101.00,101.00,101.00,101.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


In [70]:
processed_full = processed_full.fillna(0)
processed_full

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136475,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837465,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136475,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,320.00,348.00,320.00,343.139648,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038941
15996,2019-12-18,BMTR.JK,368.00,370.00,358.00,358.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,212.00,212.00,188.00,198.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,101.00,101.00,101.00,101.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


In [71]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,185.00,188.00,180.00,165.136475,4.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
1,2009-01-05,BTON.JK,83.75,83.75,83.75,79.111000,0.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
2,2009-01-05,FORU.JK,55.00,55.00,55.00,41.633144,10000.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
3,2009-01-05,GEMA.JK,28.00,28.00,28.00,21.837465,15.0,0.0,0.000000,0.0,66.666667,39.080002,0.0
4,2009-01-06,BMTR.JK,185.00,188.00,185.00,165.136475,362500.0,1.0,0.000000,0.0,66.666667,38.560001,0.0
5,2009-01-06,BTON.JK,83.75,83.75,83.75,79.111000,0.0,1.0,0.000000,0.0,66.666667,38.560001,0.0
6,2009-01-06,FORU.JK,55.00,55.00,53.00,40.119213,19500.0,1.0,-0.033966,0.0,-66.666667,38.560001,0.0
7,2009-01-06,GEMA.JK,28.00,28.00,28.00,21.837465,0.0,1.0,0.000000,0.0,-66.666667,38.560001,0.0
8,2009-01-07,BMTR.JK,189.00,189.00,185.00,168.706985,4.0,2.0,0.110721,100.0,97.052340,43.389999,0.0
9,2009-01-07,BTON.JK,83.75,83.75,83.75,79.111000,12.0,2.0,0.000000,100.0,97.052340,43.389999,0.0


In [72]:
processed_full.nunique()

date           2651
tic               4
open            872
high            893
low             893
close          1948
volume         4808
day               5
macd          10414
rsi_30         6822
cci_30         9685
vix            1441
turbulence     2397
dtype: int64

In [73]:
#memperbarui dataframe dengan bentuk harga per lot

processed_full_lot = processed_full[['open','high','low','close']].mul(100)

#drop some columns
processed_full_noOHLC = processed_full.drop(['open','high','low','close'], axis=1)
  
processed_full_lot = pd.concat([processed_full_noOHLC, processed_full_lot], axis=1)

processed_full_lot = processed_full_lot[['date', 'tic', 'open', 'high', 'low', 'close', 'volume', 'day', 'macd', 'rsi_30', 'cci_30', 'vix', 'turbulence']]

processed_full_lot

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2009-01-05,BMTR.JK,18500.0,18800.0,18000.0,16513.647461,4.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
1,2009-01-05,BTON.JK,8375.0,8375.0,8375.0,7911.100006,0.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
2,2009-01-05,FORU.JK,5500.0,5500.0,5500.0,4163.314438,10000.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
3,2009-01-05,GEMA.JK,2800.0,2800.0,2800.0,2183.746529,15.0,0.0,0.000000,0.000000,66.666667,39.080002,0.000000
4,2009-01-06,BMTR.JK,18500.0,18800.0,18500.0,16513.647461,362500.0,1.0,0.000000,0.000000,66.666667,38.560001,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,2019-12-17,GEMA.JK,32000.0,34800.0,32000.0,34313.964844,224900.0,1.0,-7.266888,41.046063,-139.035782,12.290000,7.038941
15996,2019-12-18,BMTR.JK,36800.0,37000.0,35800.0,35800.000000,31970600.0,2.0,-4.160608,48.502887,-22.494888,12.580000,5.054559
15997,2019-12-18,BTON.JK,21200.0,21200.0,18800.0,19800.000000,177800.0,2.0,-0.243493,46.726281,-76.923077,12.580000,5.054559
15998,2019-12-18,FORU.JK,10100.0,10100.0,10100.0,10100.000000,0.0,2.0,-1.345600,44.683401,-115.611439,12.580000,5.054559


<a id='4'></a>
# Bagian 5. Membangun Lingkungan Jual-Beli Saham dengan OpenAI Gym-style
Proses *training* meliputi mengamati perubahan harga saham, mengambil tindakan dan perhitungan *reward*. Dengan berinteraksi dengan lingkungan pasar, agen pada akhirnya akan memperoleh strategi perdagangan yang dapat memaksimalkan imbalan.

*Environment* pasar dibangun  menggunakan OpenAI Gym yang mensimulasikan pasar saham dengan data historis pasar.

<a id='4.1'></a>
## 5.1. Memisahkan Data
Data dibagi menjadi dua bagian, yaitu bagian pelatihan dan bagian pengujian dengan keterangan sebagai berikut:

Periode data pelatihan: 2009-01-01 hingga 2017-12-31

Periode data perdagangan: 2018-01-01 hingga 2022-08-15

In [74]:
train = data_split(processed_full_lot, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full_lot, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

8628
1976


In [75]:
train.tail()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
2155,2017-12-28,GEMA.JK,19400.0,19400.0,19400.0,18483.361816,500.0,3.0,4.955439,85.663845,85.451853,10.18,0.768637
2156,2017-12-29,BMTR.JK,58000.0,60000.0,57500.0,58438.098145,47846400.0,4.0,0.700427,51.611650,50.420800,11.04,0.652399
2156,2017-12-29,BTON.JK,11500.0,11500.0,11300.0,11300.000000,72000.0,4.0,-0.668175,47.825672,-50.397878,11.04,0.652399
2156,2017-12-29,FORU.JK,12600.0,14300.0,12400.0,12500.000000,10800.0,4.0,-19.008832,36.861538,-119.781160,11.04,0.652399
2156,2017-12-29,GEMA.JK,19400.0,19400.0,19200.0,18483.361816,6134000.0,4.0,4.784417,85.663845,75.050419,11.04,0.652399


In [76]:
trade.head()

,date,tic,open,high,low,close,volume,day,macd,rsi_30,cci_30,vix,turbulence
0,2018-01-02,BMTR.JK,58500.0,61000.0,58500.0,60419.049072,51308700.0,1.0,3.323454,54.022498,122.833904,9.77,5.289309
0,2018-01-02,BTON.JK,11300.0,11500.0,11100.0,11200.000000,72800.0,1.0,-0.763789,46.868249,-79.279279,9.77,5.289309
0,2018-01-02,FORU.JK,12000.0,12000.0,12000.0,12000.000000,100.0,1.0,-18.873467,35.964867,-122.764317,9.77,5.289309
0,2018-01-02,GEMA.JK,19200.0,20200.0,19200.0,19245.562744,1657500.0,1.0,5.003855,89.250610,121.394143,9.77,5.289309
1,2018-01-03,BMTR.JK,61000.0,61500.0,60000.0,60419.049072,51856600.0,2.0,4.876092,54.022498,148.499761,9.15,7.368325


In [77]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 4, State Space: 21


In [78]:
buy_cost_list = [0.0023] * stock_dimension
sell_cost_list = [0.0054] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

<a id='4.2'></a>
## 5.2. Lingkungan untuk Pelatihan *(Training)*



In [79]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


<a id='5'></a>
# Bagian 6: Melatih Agen DRL
* Algoritma DRL dibangun dengan menggunakan **Stable Baselines 3**. 

* FinRL mencakup algoritma DRL standar yang disempurnakan, seperti DQN, DDPG, DDPG Multi-Agen, PPO, SAC, A2C, dan TD3. FinRL juga mengizinkan pengguna untuk merancang algoritma DRL sendiri dengan mengadaptasi algoritma DRL yang telah disebutkan.

In [80]:
agent = DRLAgent(env = env_train)

**Agent Training: 5 algorithms (A2C, PPO, TD3, SAC, DDPG)**

<a id='5.1'></a>
##6.1. Agen 1: A2C


In [81]:
A2C_PARAMS = {
    "n_steps": 150, 
    "ent_coef": 0.0001, 
    "learning_rate": 0.0014,
}
model_a2c = agent.get_model("a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 150, 'ent_coef': 0.0001, 'learning_rate': 0.0014}
Using cuda device


In [82]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

--------------------------------------
| time/                 |            |
|    fps                | 363        |
|    iterations         | 100        |
|    time_elapsed       | 41         |
|    total_timesteps    | 15000      |
| train/                |            |
|    entropy_loss       | -5.69      |
|    explained_variance | -0.000111  |
|    learning_rate      | 0.0014     |
|    n_updates          | 99         |
|    policy_loss        | -2.38e+04  |
|    reward             | -245.99512 |
|    std                | 1          |
|    value_loss         | 2.68e+07   |
--------------------------------------
day: 2156, episode: 10
begin_total_asset: 100000000.00
end_total_asset: 782562011.33
total_reward: 682562011.33
total_cost: 99417737.70
total_trades: 8518
Sharpe: 0.757
--------------------------------------
| time/                 |            |
|    fps                | 360        |
|    iterations         | 200        |
|    time_elapsed       | 83         |
|    total_t

<a id='5.2'></a>
##6.2. Agen 2: PPO

In [83]:
PPO_PARAMS = {
    "n_steps": 50000,
    "ent_coef": 0.7,
    "learning_rate": 0.0016,
    "batch_size": 1750,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 50000, 'ent_coef': 0.7, 'learning_rate': 0.0016, 'batch_size': 1750}
Using cuda device


/usr/local/lib/python3.7/dist-packages/stable_baselines3/ppo/ppo.py:147: UserWarning: You have specified a mini-batch size of 1750, but because the `RolloutBuffer` is of size `n_steps * n_envs = 50000`, after every 28 untruncated mini-batches, there will be a truncated mini-batch of size 1000
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=50000 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"


In [84]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

day: 2156, episode: 30
begin_total_asset: 100000000.00
end_total_asset: 157970815.91
total_reward: 57970815.91
total_cost: 75374747.16
total_trades: 8418
Sharpe: 0.360
day: 2156, episode: 40
begin_total_asset: 100000000.00
end_total_asset: 95878156.71
total_reward: -4121843.29
total_cost: 74639943.93
total_trades: 8364
Sharpe: 0.122
-----------------------------------
| time/              |            |
|    fps             | 367        |
|    iterations      | 1          |
|    time_elapsed    | 136        |
|    total_timesteps | 50000      |
| train/             |            |
|    reward          | 0.23276515 |
-----------------------------------


<a id='5.3'></a>
##6.3. Agen 3: TD3

In [85]:
TD3_PARAMS = {
    "batch_size": 7000, 
    "buffer_size": 10000, 
    "learning_rate": 0.9  
}
model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 7000, 'buffer_size': 10000, 'learning_rate': 0.9}
Using cuda device


In [86]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000)

day: 2156, episode: 50
begin_total_asset: 100000000.00
end_total_asset: 275733131.13
total_reward: 175733131.13
total_cost: 976150.67
total_trades: 7413
Sharpe: 0.500
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 85        |
|    time_elapsed    | 100       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | -3.22e+08 |
|    critic_loss     | 3.45e+18  |
|    learning_rate   | 0.9       |
|    n_updates       | 6471      |
|    reward          | 588.4433  |
----------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 74       |
|    time_elapsed    | 231      |
|    total_timesteps | 17256    |
| train/             |          |
|    actor_loss      | 6.18e+08 |
|    critic_loss     | 1.11e+18 |
|    learning_rate   | 0.9      |
|    n_updates       | 15099    |
|   

<a id='5.4'></a>
##6.4. Agen 4: SAC

In [87]:
SAC_PARAMS = {
    "ent_coef": 1.25,
    "learning_rate": 0.0001,
    "batch_size": 4000,
    "buffer_size": 5000,
    "learning_starts": 14,
    
}
model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'ent_coef': 1.25, 'learning_rate': 0.0001, 'batch_size': 4000, 'buffer_size': 5000, 'learning_starts': 14}
Using cuda device


In [88]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=30000)

----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 61        |
|    time_elapsed    | 139       |
|    total_timesteps | 8628      |
| train/             |           |
|    actor_loss      | 3.02e+03  |
|    critic_loss     | 1.51e+06  |
|    ent_coef        | 1.25      |
|    learning_rate   | 0.0001    |
|    n_updates       | 8613      |
|    reward          | 416.00098 |
----------------------------------
day: 2156, episode: 80
begin_total_asset: 100000000.00
end_total_asset: 293352416.24
total_reward: 193352416.24
total_cost: 229471.29
total_trades: 4312
Sharpe: 0.498
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 61        |
|    time_elapsed    | 278       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | 9.57e+03  |
|    critic_loss     | 1.08e+06  |
|    ent_coef        | 1.25 

<a id='5.5'></a>
##6.5. Agen 5: DDPG

In [89]:
DDPG_PARAMS = {
    "batch_size": 1250, 
    "buffer_size": 50000, 
    "learning_rate": 0.0003
}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 1250, 'buffer_size': 50000, 'learning_rate': 0.0003}
Using cuda device


In [90]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=60000)

day: 2156, episode: 90
begin_total_asset: 100000000.00
end_total_asset: 815654484.42
total_reward: 715654484.42
total_cost: 229467.14
total_trades: 2156
Sharpe: 0.740
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 120      |
|    time_elapsed    | 71       |
|    total_timesteps | 8628     |
| train/             |          |
|    actor_loss      | -1.7e+04 |
|    critic_loss     | 2.04e+08 |
|    learning_rate   | 0.0003   |
|    n_updates       | 6471     |
|    reward          | 0.0      |
---------------------------------
----------------------------------
| time/              |           |
|    episodes        | 8         |
|    fps             | 106       |
|    time_elapsed    | 161       |
|    total_timesteps | 17256     |
| train/             |           |
|    actor_loss      | -9.99e+03 |
|    critic_loss     | 6.75e+07  |
|    learning_rate   | 0.0003    |
|    n_updates       | 15099     |
|    r

<a id='6'></a>
# Bagian 7: Melakukan Jual-Beli

<a id='6.1'></a>
## 7.1. Performa di Dalam Sampel

Asumsikan modal awal adalah Rp100.000.000.

In [91]:
data_risk_indicator = processed_full_lot[(processed_full_lot.date<TRAIN_END_DATE) & (processed_full_lot.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [92]:
insample_risk_indicator.vix.describe()

count    2157.000000
mean       18.666583
std         7.813415
min         9.140000
25%        13.370000
50%        16.299999
75%        21.540001
max        56.650002
Name: vix, dtype: float64

In [93]:
insample_risk_indicator.vix.quantile(0.996)

48.84047967529307

In [94]:
insample_risk_indicator.turbulence.describe()

count      2157.000000
mean        290.374781
std       13329.686848
min           0.000000
25%           0.250728
50%           1.145774
75%           3.447181
max      619080.769224
Name: turbulence, dtype: float64

In [95]:
insample_risk_indicator.turbulence.quantile(0.996)

60.363542710264426

<a id='6.2'></a>
##7.2. Performa di Luar Sampel

FinRL dapat diatur dengan cara melakukan pelatihan ulang secara berkala, misalnya, pelatihan ulang setiap tiga bulan, bulanan, atau mingguan. 

Pada notebook ini, *hyperparameter* hanya diatur satu kali dengan menggunakan data sampel selama periode yang telah ditetapkan. Sehingga, terjadi beberapa peluruhan alfa seiring dengan perpanjangan tanggal perdagangan.

Banyak *hyperparameter* – mis. learning rate, mempengaruhi proses pembelajaran dan biasanya ditentukan dengan menguji beberapa variasi.

In [96]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

<a id='6.3'></a>
##7.3. Hasil Jual-Beli Untuk Setiap Agen Berupa Rangkuman Aksi

<a id='6.3.1'></a>
###7.3.1. Agen 1: A2C

In [97]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym)

hit end!


<a id='6.3.2'></a>
###7.3.2. Agen 2: PPO

In [98]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym)

hit end!


<a id='6.3.3'></a>
###7.3.3. Agen 3: TD3

In [99]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym)

hit end!


<a id='6.3.4'></a>
###7.3.4. Agen 4: SAC

In [100]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

hit end!


<a id='6.3.5'></a>
###7.3.5. Agen 5: DDPG

In [101]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym)

hit end!


<a id='7'></a>
# Bagian 8: Performa Backtesting
Backtesting memiliki peran kunci dalam mengevaluasi kinerja strategi perdagangan. Alat backtesting otomatis lebih disukai karena dapat meminimalisir kesalahan manusia. Backtesting dapat dilakukan dengan menggunakan paket Quantopian pyfolio untuk menguji strategi perdagangan pada notebook ini. Backtesting tersebut mudah digunakan dan terdiri dari berbagai plot yang memberikan gambaran komprehensif tentang kinerja strategi perdagangan.

<a id='7.1'></a>
## 8.1 Status BackTesting


###8.1.1 Agen A2C

In [102]:
print("===========Get Backtest Results of A2C Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_a2c)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_a2c_"+now+'.csv')

===========Get Backtest Results of A2C Model===========
Annual return         -0.158699
Cumulative returns    -0.287342
Annual volatility      0.256900
Sharpe ratio          -0.545908
Calmar ratio          -0.370538
Stability              0.609616
Max drawdown          -0.428292
Omega ratio            0.902954
Sortino ratio         -0.786016
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.033683
Daily value at risk   -0.032923
dtype: float64


###8.1.2 Agen ppo

In [103]:
print("===========Get Backtest Results of PPO Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ppo_"+now+'.csv')

===========Get Backtest Results of PPO Model===========
Annual return         -0.044509
Cumulative returns    -0.085386
Annual volatility      0.183436
Sharpe ratio          -0.155603
Calmar ratio          -0.252146
Stability              0.160706
Max drawdown          -0.176521
Omega ratio            0.967023
Sortino ratio         -0.211618
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.354273
Daily value at risk   -0.023224
dtype: float64


###8.1.3 Agen TD3

In [104]:
print("===========Get Backtest Results of TD3 Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_td3)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_td3_"+now+'.csv')

===========Get Backtest Results of TD3 Model===========
Annual return         -0.091480
Cumulative returns    -0.171444
Annual volatility      0.309080
Sharpe ratio          -0.156168
Calmar ratio          -0.216106
Stability              0.152868
Max drawdown          -0.423313
Omega ratio            0.971978
Sortino ratio         -0.226429
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.191498
Daily value at risk   -0.039132
dtype: float64


###8.1.4 Agen SAC

In [105]:
print("===========Get Backtest Results of SAC Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_sac)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_sac_"+now+'.csv')

===========Get Backtest Results of SAC Model===========
Annual return         -0.136565
Cumulative returns    -0.250123
Annual volatility      0.411194
Sharpe ratio          -0.152031
Calmar ratio          -0.290049
Stability              0.555947
Max drawdown          -0.470835
Omega ratio            0.972417
Sortino ratio         -0.223032
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.214177
Daily value at risk   -0.052054
dtype: float64


###8.1.5 Agen DDPG

In [106]:
print("===========Get Backtest Results of DDPG Model===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ddpg)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+RESULTS_DIR+"/perf_stats_all_ddpg_"+now+'.csv')

===========Get Backtest Results of DDPG Model===========
Annual return          0.241883
Cumulative returns     0.529072
Annual volatility      0.332245
Sharpe ratio           0.819511
Calmar ratio           1.249768
Stability              0.903308
Max drawdown          -0.193542
Omega ratio            1.231156
Sortino ratio          1.231002
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.421368
Daily value at risk   -0.040779
dtype: float64


###8.1.6. Baseline Stats

In [107]:
#baseline stats
print("===========Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^JKII", 
        start = df_account_value_a2c.loc[0,'date'],
        end = df_account_value_a2c.loc[len(df_account_value_a2c)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')

===========Get Baseline Stats===========
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (478, 8)
Annual return         -0.041972
Cumulative returns    -0.078113
Annual volatility      0.185511
Sharpe ratio          -0.138819
Calmar ratio          -0.175216
Stability              0.093175
Max drawdown          -0.239543
Omega ratio            0.977818
Sortino ratio         -0.189296
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.900451
Daily value at risk   -0.023474
dtype: float64
